In [1]:
# make the file headers correct
# https://neo4j.com/docs/operations-manual/current/tools/import/file-header-format/

In [1]:
import os
import pickle
%matplotlib inline
import pandas as pd
import seaborn as sns
import shelve
import re
from collections import defaultdict, Counter
from tqdm import tqdm
import requests
from pyquery import PyQuery as pq

In [2]:
# https://biolink.github.io/biolink-model/docs/NamedThing.html
nodes = pd.read_csv("nodes_biolink.csv")
nodes.head()

,ID,label,TYPE,bl_type
0,C0038195,State Medicine,Activities & Behaviors,activity_and_behavior
1,C1290952,Taking medication,Activities & Behaviors,activity_and_behavior
2,C0085092,Parenting behavior,Activities & Behaviors,activity_and_behavior
3,C1096771,Murderer,Activities & Behaviors,activity_and_behavior
4,C1299651,Serious reportable event,Activities & Behaviors,activity_and_behavior


In [3]:
del nodes['TYPE']

In [4]:
nodes.ID = "UMLS:" + nodes.ID
nodes['category:STRING'] = nodes.bl_type
nodes['id:STRING'] = nodes.ID

In [5]:
nodes.rename(columns = {'ID': ':ID', 'label': 'name:STRING', 'bl_type': ':LABEL'}, inplace=True)

In [6]:
nodes.head()

,:ID,name:STRING,:LABEL,category:STRING,id:STRING
0,UMLS:C0038195,State Medicine,activity_and_behavior,activity_and_behavior,UMLS:C0038195
1,UMLS:C1290952,Taking medication,activity_and_behavior,activity_and_behavior,UMLS:C1290952
2,UMLS:C0085092,Parenting behavior,activity_and_behavior,activity_and_behavior,UMLS:C0085092
3,UMLS:C1096771,Murderer,activity_and_behavior,activity_and_behavior,UMLS:C1096771
4,UMLS:C1299651,Serious reportable event,activity_and_behavior,activity_and_behavior,UMLS:C1299651


In [7]:
nodes.to_csv("nodes_neo4j.csv", index=False)

In [8]:
###### EDGES

In [ ]:
# https://biolink.github.io/biolink-model/docs/Association.html
edges = pd.read_csv('edges_biolink.csv')

In [ ]:
edges.START_ID = "UMLS:" + edges.START_ID
edges.END_ID = "UMLS:" + edges.END_ID

In [ ]:
edges.head()

In [ ]:
edges.bl_pred = edges.bl_pred.str.lower()
edges.rename(columns = {'START_ID': ':START_ID', 'END_ID': ':END_ID', 
                        'bl_pred': ':TYPE', 'NEG': 'negated'}, inplace=True)

In [ ]:
edges['is_defined_by'] = "semmeddb"
edges['relation'] = "semmeddb:" + edges[":TYPE"].str.lower()
edges['provided_by'] = "semmeddb_sulab"

In [ ]:
edges.head()

In [ ]:
edges.to_csv("edges_neo4j.csv", index=False)